![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [6]:
import os
from pathlib import Path

def list_files_with_max_depth(path, max_depth=0):
    path = Path(path)
    if not path.is_dir():
        print(f"错误：路径 '{path}' 不是一个目录。")
        return []

    results = []
    
    # 获取起始路径的目录层级数量
    start_depth = len(path.parts)

    # 遍历目录树，topdown=True (默认)
    for root, dirs, files in os.walk(path):
        current_path = Path(root)
        
        # 1. 计算当前目录的深度
        # 深度 = 当前路径的层级数 - 起始路径的层级数
        depth = len(current_path.parts) - start_depth

        # 2. 检查并限制深度
        if max_depth >= 0 and depth > max_depth:
            # 如果超过最大深度，则通过清空 dirs 列表，阻止 os.walk 进一步进入子目录
            dirs[:] = [] 
            continue # 跳过对当前超深目录的处理

        # 3. 记录当前目录本身 (除非是起始路径且深度限制为0)
        # 我们始终记录当前路径，除非它是根目录且 max_depth=0，此时我们只关注其内容
        if not (depth == 0 and max_depth == 0):
             results.append((str(current_path), 'Directory', depth))

        # 4. 记录当前目录下的文件
        for file in files:
            file_path = current_path / file
            # 这里的深度仍然是父目录的深度 (depth)，
            # 如果你想用层级表示，可以设置为 depth + 1
            results.append((str(file_path), 'File', depth + 1)) 
           
    return results

target_dir = "/Storage/footprint_data"

MAX_DEPTH = 1
print(f"### max_depth = {MAX_DEPTH} ###")
scanned_items = list_files_with_max_depth(target_dir, max_depth=MAX_DEPTH)

# 3. 打印结果
for path_str, item_type, depth in scanned_items:
    # 打印格式：[类型] [深度] 路径名
    # 使用缩进表示层级
    indentation = "  " * depth
    print(f"{indentation}[{item_type}]\t{path_str}")


In [30]:
import os
import shutil
import uuid
from pathlib import Path

def test_file_operations(base_path: str):
    """
    测试在指定路径下创建文件夹、文件、写入和读取的权限。

    Args:
        base_path: 要测试的根目录路径。
    """
    # 确保 base_path 是一个 Path 对象，并处理路径分隔符
    test_root = Path(base_path)

    # 使用 UUID 生成一个唯一的测试名称，避免干扰现有文件
    unique_id = str(uuid.uuid4())
    
    # 1. 定义测试路径
    test_dir_name = f"test_dir_abcdef"
    test_file_name = f"test_file_abcdef.txt"
    
    test_dir_path = test_root / test_dir_name
    test_file_path = test_root / test_file_name
    
    test_content = "This is a test line for read/write verification."
    
    print(f"--- 开始测试 ---")
    print(f"开始测试路径: {test_root}")
    print(f"    - 目标文件夹: {test_dir_path}")
    print(f"    - 目标文件: {test_file_path}")
    
    # --- 异常处理和清理函数 ---
    def cleanup():
        pass
    # --- 执行测试步骤 ---
    try:
        # 1. 文件夹创建测试
        print("\n--- 1. 文件夹创建测试 ---")
        try:
            test_dir_path.mkdir(parents=True, exist_ok=False)
            print(f"成功创建文件夹: {test_dir_path}")
        except FileExistsError:
            print("文件夹已存在，跳过创建测试。")
        except PermissionError:
            print(f"失败：没有权限在 {test_root} 创建文件夹。")
            return # 无法创建，后续测试也可能失败，直接返回
        except Exception as e:
            print(f"创建文件夹失败。错误: {e}")
            return

        # 2. 文件创建和写入测试
        print("\n--- 2. 文件创建和写入测试 ---")
        try:
            # 使用 'w' 模式创建文件并写入
            with open(test_file_path, 'w', encoding='utf-8') as f:
                f.write(test_content)
            print(f"成功创建并写入文件: {test_file_path}")
        except PermissionError:
            print(f"失败：没有权限在 {test_root} 创建/写入文件。")
            return
        except Exception as e:
            print(f"文件创建/写入失败。错误: {e}")
            return

        # 3. 文件读取测试
        print("\n--- 3. 文件读取测试 ---")
        try:
            # 使用 'r' 模式读取文件
            with open(test_file_path, 'r', encoding='utf-8') as f:
                read_content = f.read()
            
            if read_content == test_content:
                print(f"成功读取文件。内容匹配。")
                print(f"   [读取内容]: '{read_content}'")
            else:
                print(f"失败：读取的内容不匹配。")
                print(f"   [预期内容]: '{test_content}'")
                print(f"   [实际内容]: '{read_content}'")
                
        except PermissionError:
            print(f"失败：没有权限读取文件 {test_file_path}。")
        except FileNotFoundError:
            print(f"失败：文件 {test_file_path} 不存在，无法读取。")
        except Exception as e:
            print(f"文件读取失败。错误: {e}")


    except Exception as e:
        print(f"\n致命错误：测试过程中发生意外错误: {e}")

    finally:
        # 4. 清理创建的文件和文件夹
        cleanup()

# =================================================================
# 目标路径
# =================================================================
target_path = "/QuantConnect/research-cloud/airlock"

# 运行测试
test_file_operations(target_path)